In [3]:
import numpy as np
import sys

In [4]:
REGISTERS_COUNT = 8
IP_ADDRESS = 0  # instructions pointer
SP_ADDRESS = 1  # stack pointer

INSTRUCTION_SIZE = 3  # количество ячеек, занимаемое одной инструкцией

INSTRUCTIONS_DICT = {'MOV':0, 'ADD':1, 'JUMP':2, 'STOP':3, 'READ':4, 'PRINT':5, 'FBEG':6, 'FEND':7, 'PUSH':8, 'POP':9, 'CALL':10, 'TOP':11, 'SUB':12, 'ASSIGN':13, 'IFNIL':14, 'PUSHREG':15, 'PUTC':16}

In [5]:
class VirtualMachine:
    
    def __init__(self, size):
        self.memory = np.zeros(size, dtype=np.int)
        self.memory[IP_ADDRESS] = REGISTERS_COUNT  # первые ячейки памяти выделены для регистров
        # стек начинается в последней ячейке памяти и растёт к началу выделенного участка
        self.memory[SP_ADDRESS] = size - 1 
        
        # Для поддержки функций 
        
        # Хранит адрес первой инструкции функции с числовым названием (ключ)
        self.func_start_address = {}
        # флаг, отвечающий за считывание объявления функции
        self.func_prototype_reading = False 
    
    # Работа с отдельными ячейками памяти
    
    def read_from_address(self, address):
        return self.memory[address]
    
    def write_value_to_address(self, address, value):
        self.memory[address] = value
        
    
    # Считывание программы из входного numpy массива / обычного массива
    # программа содержит только инструкции - записываются прямо за регистрами
    
    def read_program_from_array(self, program):
        self.func_start_address = {}
        self.func_prototype_reading = False 
        
        for code_idx in range(len(program)):
            self.memory[REGISTERS_COUNT + code_idx] = program[code_idx]
    
    def read_program_from_file(self, filename):
        with open(filename) as f:
            program_str = f.readlines()
        program_line = program_str[0]
        program_line = program_line.split()
        program = [int(num) for num in program_line]
        
        self.read_program_from_array(program)
            
    
    # Работа с инструкциями
    
    # переход к следующей инструкции
    def move_to_next_instruction(self):
        self.write_value_to_address(IP_ADDRESS, self.read_from_address(IP_ADDRESS) + INSTRUCTION_SIZE)
        
    def get_instruction_value(self, address):
        return self.memory[address:address + INSTRUCTION_SIZE]
    
    def get_instruction_id(self, instruction_name):
        return INSTRUCTIONS_DICT[instruction_name]
    
    
    def mov(self, dest_addr, src_addr):
        """
        копирование данных из регистра номер src_addr в регистр номер dest_addr
        Инструкция записывается: [1, dest_addr, src_addr]
        """
        data = self.read_from_address(src_addr)
        self.write_value_to_address(dest_addr, data)
        self.move_to_next_instruction() ## возможно, тут понадобится какое-нибудь условие
        
    def add(self, dest_addr, addition_addr):
        """
        добавление данных из регистра номер addition_addr к данным из регистра dest_addr
        Инструкция записывается: [1, dest_addr, addition_addr]
        """
        addition_value = self.read_from_address(addition_addr)
        dest_value = self.read_from_address(dest_addr)
        
        self.write_value_to_address(dest_addr, dest_value + addition_value)
        self.move_to_next_instruction() ## возможно, тут понадобится какое-нибудь условие
        
    def jump(self, dest_addr):
        """
        instruction_pointer перемещается на адрес dest_addr
        Инструкция записывается: [2, 0, dest_addr]
        """
        self.write_value_to_address(IP_ADDRESS, dest_addr)
        
    def read(self, dest_addr):
        """
        dest_addr - номер регистра, в который будет записано считанное число
        Инструкция записывается: [4, 0, dest_addr]
        """
        value = int(input())
        self.write_value_to_address(dest_addr, value)
        self.move_to_next_instruction() 
        
    def print_reg(self, reg_addr):
        """
        reg_addr - номер регистра, содержимое которого необходимо вывести на экран
        Инструкция записывается: [5, 0, reg_addr]
        """
        print(self.read_from_address(reg_addr))
        self.move_to_next_instruction()
        
    def function_begin(self, name_num):
        """
        принимает числовое "имя" функции
        Инструкция записывается: [6, 0, name_num]
        """
        self.func_prototype_reading = True
        self.func_start_address[name_num] = self.read_from_address(IP_ADDRESS) + INSTRUCTION_SIZE
        self.move_to_next_instruction()
        
    def function_end(self):
        """
        Определяет окончание определеня функции
        Инструкция записывается: [7, 0, 0]
        """
        if self.func_prototype_reading:
            self.func_prototype_reading = False
            self.move_to_next_instruction()
        else:  # это было не определение функции, а вызов
            # Выходим из функции - переход на следующую инструкцию, адрес которой лежит в стеке
            self.write_value_to_address(IP_ADDRESS, self.read_from_address(self.read_from_address(SP_ADDRESS) + 1))
            self.pop()
        
    def push(self, value, is_move_to_next_instruction=False):
        """
        Кладёт на стек число value
        Инструкция записывается: [8, 0, value]
        """
        self.write_value_to_address(self.read_from_address(SP_ADDRESS), value)
        self.write_value_to_address(SP_ADDRESS, self.read_from_address(SP_ADDRESS) - 1)
        if is_move_to_next_instruction:
            self.move_to_next_instruction()
    
    def pop(self, is_move_to_next_instruction=False):
        """
        "Удаляет" верхнее значение со стека (увеличивает stack pointer)
        Инструкция записывается: [9, 0, 0]
        """
        self.write_value_to_address(SP_ADDRESS, self.read_from_address(SP_ADDRESS) + 1)
        if is_move_to_next_instruction:
            self.move_to_next_instruction()
        
    def call(self, name_num):
        """
        Вызывает функцию с числовым именем name_num
        Инструкция записывается: [10, 0, name_num]
        """
        # Кладём в стек адрес следующей за вызовом функции инструкции
        self.push(self.read_from_address(IP_ADDRESS) + INSTRUCTION_SIZE)
        self.write_value_to_address(IP_ADDRESS, self.func_start_address[name_num])
        
    def top(self, reg_addr):
        """
        Копирует информацию, записанную в верхушке стека, в регистр reg_addr
        Инструкция записывается: [11, 0, reg_addr]
        """
        self.write_value_to_address(reg_addr, self.read_from_address(self.read_from_address(SP_ADDRESS) + 1))
        self.move_to_next_instruction()
        
    def sub(self, dest_addr, subtract_addr):
        """
        Вычитает значение, лежащее в subtract_addr из dest_addr
        Инструкция записывается: [12, dest_addr, subtract_addr]
        """
        subtract_value = self.read_from_address(subtract_addr)
        dest_value = self.read_from_address(dest_addr)
        
        self.write_value_to_address(dest_addr, dest_value - subtract_value)
        self.move_to_next_instruction() ## возможно, тут понадобится какое-нибудь условие
        
    def assign(self, dest_addr, value):
        """
        Записывает в ячейку с адресом dest_addr значение value
        Инструкция записывается: [13, dest_addr, value]
        """
        self.write_value_to_address(dest_addr, value)
        self.move_to_next_instruction()
        
    def ifnil(self, reg_addr, jump_addr):
        """
        Если значение, лежащее по адресу reg_addr равно нулю, переносит IP на адрес jump_addr
        Иначе - просто переходит на следующую инструкцию
        Инструкция записывается: [14, reg_addr, jump_addr]
        """
        reg_value = self.read_from_address(reg_addr)
        if reg_value == 0:
            self.jump(jump_addr)
        else:
            self.move_to_next_instruction()
            
    def push_reg(self, reg_addr):
        """
        Кладёт на стек значение, лежащее по адресу reg_addr
        Инструкция записывается: [15, 0, reg_addr]
        """
        reg_value = self.read_from_address(reg_addr)
        self.push(reg_value, is_move_to_next_instruction=True)
        
    def put_char(self, char_num):
        """
        Выводит на экран символ, соответствующий числу char_num
        Инструкция записывается: [16, 0, char_num]
        """
        print(chr(char_num), end='')
        self.move_to_next_instruction()
        
        
    # Интерпретация команд
    
    def interpret_current_comand(self):
        """
        интерпретирует команду, на которую указывает IP
        returns: True, если нужно продолжать выполнение
                 False, если выполнение программы необходимо прекратить
        """
        instruction_value = self.get_instruction_value(self.read_from_address(IP_ADDRESS))
        #print(instruction_value)
        
        instruction_id = instruction_value[0]
        arg_first = instruction_value[1]
        arg_second = instruction_value[2]
        
        # Если сейчас считываем определение функции, ничего выполнять не нужно, просто переходим дальше
        if self.func_prototype_reading and not instruction_id == INSTRUCTIONS_DICT['FEND']:
            self.move_to_next_instruction()
            return True
        
        if instruction_id == INSTRUCTIONS_DICT['MOV']:  # 0
            self.mov(arg_first, arg_second)
            return True
        
        if instruction_id == INSTRUCTIONS_DICT['ADD']:  # 1
            self.add(arg_first, arg_second)
            return True
        
        if instruction_id == INSTRUCTIONS_DICT['JUMP']:  # 2
            self.jump(arg_second)
            return True
        
        if instruction_id == INSTRUCTIONS_DICT['STOP']:  # 3
            return False
        
        if instruction_id == INSTRUCTIONS_DICT['READ']:  # 4
            self.read(arg_second)
            return True
        
        if instruction_id == INSTRUCTIONS_DICT['PRINT']:  # 5
            self.print_reg(arg_second)
            return True
        
        if instruction_id == INSTRUCTIONS_DICT['FBEG']:  # 6
            self.function_begin(arg_second)
            return True
        
        if instruction_id == INSTRUCTIONS_DICT['FEND']:  # 7
            self.function_end()
            return True 
        
        if instruction_id == INSTRUCTIONS_DICT['PUSH']:  # 8
            self.push(arg_second, is_move_to_next_instruction=True)
            return True
        
        if instruction_id == INSTRUCTIONS_DICT['POP']:  # 9
            self.pop(is_move_to_next_instruction=True)
            return True
        
        if instruction_id == INSTRUCTIONS_DICT['CALL']:  # 10
            self.call(arg_second)
            return True
        
        if instruction_id == INSTRUCTIONS_DICT['TOP']:  # 11
            self.top(arg_second)
            return True
        
        if instruction_id == INSTRUCTIONS_DICT['SUB']:  # 12
            self.sub(arg_first, arg_second)
            return True
        
        if instruction_id == INSTRUCTIONS_DICT['ASSIGN']:  # 13
            self.assign(arg_first, arg_second)
            return True
        
        if instruction_id == INSTRUCTIONS_DICT['IFNIL']:  # 14
            self.ifnil(arg_first, arg_second)
            return True
        
        if instruction_id == INSTRUCTIONS_DICT['PUSHREG']:  # 15
            self.push_reg(arg_second)
            return True
        
        if instruction_id == INSTRUCTIONS_DICT['PUTC']:  # 16
            self.put_char(arg_second)
            return True
        
    def run(self):
        """
        Запускается работа машины.
        Машина работает с выделенным участком памяти, пока не будет прочитана инструкция STOP
        """
        self.memory[IP_ADDRESS] = REGISTERS_COUNT
        self.memory[SP_ADDRESS] = len(self.memory) - 1
        
        while self.interpret_current_comand():
            continue

In [6]:
vm = VirtualMachine(200)

# Попробуем написать программу удвоения числа

# fbeg (0) 0      фя удвоения числа
# read (0) 2
# sub 3 2
# add 2 3
# print_reg (0) 2
# fend (0) 0     окончание фии удвоения числа
# ifnil 4 32    если значение в регистре 4 равно 0 перейти на 32
# call (0) 0     вызов фии удвоения числа
# call (0) 0     вызов фии удвоения числа (это 32)
# stop
program = [6, 0, 0, 4, 0, 2, 0, 3, 2, 12, 2, 3, 5, 0, 2, 7, 0, 0, 14, 4, 32, 10, 0, 0, 10, 0, 0, 3]
vm.read_program_from_array(program)

# нужна команда print из регистра (+ возможно, print строку)
# Назначение и приглашение ввести строку, возможно, нужно написать в захардкоженной команде print hello?

print(vm.memory)

vm.run()

print(vm.memory)



[  8 199   0   0   0   0   0   0   6   0   0   4   0   2   0   3   2  12
   2   3   5   0   2   7   0   0  14   4  32  10   0   0  10   0   0   3
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0]
8
0
[ 35 199   0   8   0   0   0   0   6   0   0   4   0   2   0   3   2  12
   2   3   5   0   2   7   0   0  14   4  32  10   0   0  10   0   0   3
   0   0   0   0   0   0   0   0   0 

In [7]:
# Теперь аналогично нужно написать вычисление числа Фибоначчи

## fbeg (0) 42    фя вычисления числа Фибоначчи (парметр лежит в r1 - адрес 2)
## ifnil 2(r1) (...)addr([ret1])   
## assign 3(r2) 1
## sub 2(r1) 3(r2)
## ifnil 2(r1) (...)addr([ret1])
## pushreg (0) 2(r1)
## call (0) 42    вызов функции вычисления числа Фибоначчи для n - 1
## top (0) 2(r1)
## pop (0) (0)
## assign 3(r2) 1
## sub 2(r1) 3(r2)
## pushreg (0) 7(r6)
## call (0) 42   вызов функции вычисления числа Фибоначчи для n - 2
## top (0) 6(r5)
## pop (0) 0
## add 7(r6) 6(r5)
## jump (0) (...)addr([finish])
## assign 7(r6) 1 [ret1]
## fend (0) (0) [finish]  конец фии вычисления числа Фибоначчи, ответ лежит в 7(r6)
## read (0) 2(r1)
## call (0) 42
## print_reg (0) 7(r6)
## stop

program = [6, 0, 42,
           14, 2, 59,
           13, 3, 1,
           12, 2, 3,
           14, 2, 59,
           15, 0, 2,
           10, 0, 42,
           11, 0, 2,
           9, 0, 0,
           13, 3, 1,
           12, 2, 3,
           15, 0, 7,
           10, 0, 42,
           11, 0, 6,
           9, 0, 0,     # pop (0) 0
           1, 7, 6,     # add 7(r6) 6(r5)
           2, 0, 62,    # jump (0) (...)addr([finish])
           13, 7, 1,    # assign 7(r6) 1 [ret1] - 59
           7, 0, 0,     # fend (0) (0) [finish] - 62
           4, 0, 2,     # read (0) 2(r1)
           10, 0, 42,   # call (0) 42
           5, 0, 7,     # print_reg (0) 7(r6)
           3, 0, 0      # stop
          ]

vm = VirtualMachine(300)

vm.read_program_from_array(program)

#print(vm.memory)

vm.run()

#print(vm.memory)


3
3


In [8]:
REGISTERS_DICT = {'ip':0, 'sp':1, 'r1':2, 'r2':3, 'r3':4, 'r4':5, 'r5':6, 'r6':7}    

class Assembler:
    
    def __init__(self):
        self.program_text = []  # Каждый элемент массива будет соответствовать одной строке кода
    
    def read_file(self, filename):
        with open(filename) as f:
            self.program_text = f.readlines()
            self.program_text = [line.strip() for line in self.program_text]
            
    def create_putcs_for_putstr(self, putstr_instruction):
        """
        Для инструкции, содержащей PUTSTR возвращает список инструкций, содержащий PUTC
        """
        # Обработка лейблов
        has_label = False
        # Проверить, есть ли в строке label
        if putstr_instruction.find(':') != -1 and putstr_instruction.find(':') < putstr_instruction.find('PUTSTR'):
            label = putstr_instruction[:putstr_instruction.find(':') + 1]
            has_label = True
        
        # Список символов, которые необходимо вывести
        chars = list(putstr_instruction[putstr_instruction.find('PUTSTR') + len('PUTSTR '):])
        
        chars.append('\n')  
        
        result_instructions = []
        for char in chars:
            if has_label:
                # Добавляем инструкцию [label: PUTC номер_символа]
                result_instructions.append(label + ' PUTC ' + str(ord(char)))
                has_label = False
            # Добавляем инструкцию [PUTC номер_символа]
            result_instructions.append('PUTC ' + str(ord(char)))
            
        return result_instructions
            
    def replace_all_putstr_with_putcs(self):
        """
        В массиве program_text все вхождения инструкций с PUTSTR 
        заменяет на вхождение последовательности команд PUTC
        """
        
        new_program_text = []
        
        for i, line in enumerate(self.program_text):
            
            if line.find('PUTSTR') != -1:  # в строке есть инструкция PUTSTR
                new_instructions = self.create_putcs_for_putstr(line)
                for instruction in new_instructions:
                    new_program_text.append(instruction)
                continue
                
            new_program_text.append(line)
                
        self.program_text = new_program_text
    
            
    def replace_labels_with_cell_numbers(self):
        """
        В массиве program_text заменяет лейблы на соответствующие номера ячеек 
        (каким они будут соответствовать в байт-коде)
        """
        label_to_num = {}
        
        result = []
        
        for i, line in enumerate(self.program_text):
            if line.find(':') != -1:  # в строке есть : -> в строке уканан лейбл
                label = line[:line.find(':')]
                label_to_num[label] = i  # соответствие label -> номер строки, в которой встретился
                line = line[line.find(':') + 1:]
            line = line.strip()
            result.append(line)
            
        lines = result
        result = []
        
        for line in lines:
            for label, num in label_to_num.items():
                line = line.replace(label, str(num * INSTRUCTION_SIZE + REGISTERS_COUNT))
            result.append(line)
        self.program_text = result
        
        
    def convert_to_static(self, string_parameter):
        """
        Возвращает либо адрес регистра, которому соответствует переданная строка,
        либо статическое число
        """
        if string_parameter in REGISTERS_DICT:
            return REGISTERS_DICT[string_parameter]
        return int(string_parameter)
        
    def convert_string_to_code(self, string):
        """
        По входной строке генерирует соответствующий ей байт-код.
        - заменяет название инструкции на соответствующий id;
        - делает все инструкции трёхсложными (дополняет нулями)
        - заменяет названия регистров на соответствующие им адреса в памяти
        
        Возвращает массив из 3х чисел.
        """
        
        # Массив параметров, содержащихся в строке
        string_params = string.split()
        
        # первый параметр заменяем на соответствующий id инструкции
        string_params[0] = INSTRUCTIONS_DICT[string_params[0]]
        
        # делаем все инструкции трехсложными
        if len(string_params) < 3:
            if len(string_params) == 2:
                string_params.append(self.convert_to_static(string_params[1]))
                string_params[1] = 0
            if len(string_params) == 1:
                string_params.append(0)
                string_params.append(0)
        else:
            string_params[1] = self.convert_to_static(string_params[1])
            string_params[2] = self.convert_to_static(string_params[2])
        
        return string_params
    
    
    def generate_bytecode(self):
        """
        Считанный текст программы преобразует в байт-код, возвращает соответствующий массив
        """
        self.replace_all_putstr_with_putcs()
        self.replace_labels_with_cell_numbers()
        
        byte_code = []
        
        for instruction in self.program_text:
            instr_byte_code = self.convert_string_to_code(instruction)
            for code in instr_byte_code:
                byte_code.append(code)
        
        return byte_code
    
    
    def assembly(self, program_file, bytecode_file):
        """
        Принимает 
        - путь до файла с программой, написанной вспомогательным языком,
        - путь до файла, в который будет записан полученный бинарный файл с программой
        """
        
        self.read_file(program_file)
        bytecode = self.generate_bytecode()
        # записать bytecode в файл
        with open(bytecode_file, 'w') as outfile:
            for code in bytecode:
                outfile.write("%d " % code)
    

In [13]:
asm = Assembler()
asm.assembly("fib.asm", "fib.bin")

vm = VirtualMachine(500)
vm.read_program_from_file("fib.bin")
vm.run()

Эта программа возвращает n-й элемент последовательности Фибоначчи!
Введите число n:
8
Результат:
34


In [10]:
def get_reg_name(reg_addr):
    """
    По адресу регистра (числу от 0 до 7) возвращает его строковое имя
    """
    for name, addr in REGISTERS_DICT.items():
        if addr == reg_addr:
            return name
        
class Disassembler:
    
    def __init__(self):
        self.program_code = []
        
        # адрес строки - label этой строки
        self.address_to_label = {}
        
        self.labels_counter = 0
    
    # Как и VirtualMachine умеет считывать программу из bin файла
    def read_program_from_file(self, filename):
        with open(filename) as f:
            program_str = f.readlines()
        program_line = program_str[0]
        program_line = program_line.split()
        self.program_code = [int(num) for num in program_line]

    def convert_code_to_string(self, instruction, arg_first, arg_second, use_labels=True):
        """
        Конфертирует команду состоящую из 3х чисел в ассемблерную инструкцию
        Если выставлен флаг use_labels заменяет в переходах ip на новые адреса эти самые адреса на названия лейблов,
        лейблы при этом запоминает
        """
        if instruction == 0:  # MOV
            string_result = 'MOV ' + get_reg_name(arg_first) + ' ' + get_reg_name(arg_second)
            
        if instruction == 1:  # ADD
            string_result = 'ADD ' + get_reg_name(arg_first) + ' ' + get_reg_name(arg_second)
            
        if instruction == 2:  # JUMP
            if use_labels:
                address = (arg_second - REGISTERS_COUNT) / INSTRUCTION_SIZE
                if address in self.address_to_label:
                    string_result = 'JUMP ' + self.address_to_label[address]
                else:
                    self.address_to_label[address] = 'label' + str(self.labels_counter)
                    string_result = 'JUMP ' + 'label' + str(self.labels_counter)
                    self.labels_counter += 1
            else:
                string_result = 'JUMP ' + str(arg_second)
            
        if instruction == 3:  # STOP
            string_result = 'STOP'
            
        if instruction == 4:  # READ    
            string_result = 'READ ' + get_reg_name(arg_second)
            
        if instruction == 5:  # PRINT
            string_result = 'PRINT ' + get_reg_name(arg_second)
            
        if instruction == 6:  # FBEG
            string_result = 'FBEG ' + str(arg_second)
            
        if instruction == 7:  # FEND
            string_result = 'FEND'
            
        if instruction == 8:  # PUSH
            string_result = 'PUSH ' + str(arg_second)
            
        if instruction == 9:  # POP
            string_result = 'POP'
            
        if instruction == 10:  # CALL
            string_result = 'CALL ' + str(arg_second)
            
        if instruction == 11:  # TOP
            string_result = 'TOP ' + get_reg_name(arg_second)
            
        if instruction == 12:  # SUB
            string_result = 'SUB ' + get_reg_name(arg_first) + ' ' + get_reg_name(arg_second)
            
        if instruction == 13:  # ASSIGN
            string_result = 'ASSIGN ' + get_reg_name(arg_first) + ' ' + str(arg_second)
            
        if instruction == 14:  # IFNIL
            if use_labels:
                address = (arg_second - REGISTERS_COUNT) / INSTRUCTION_SIZE
                if address in self.address_to_label:
                    string_result = 'IFNIL ' + get_reg_name(arg_first) + ' ' + self.address_to_label[address]
                else:
                    self.address_to_label[address] = 'label' + str(self.labels_counter)
                    string_result = 'IFNIL ' + get_reg_name(arg_first) + ' label' + str(self.labels_counter)
                    self.labels_counter += 1
            else:
                string_result = 'IFNIL ' + get_reg_name(arg_first) + ' ' + str(arg_second)
            
        if instruction == 15:  # PUSHREG
            string_result = 'PUSHREG ' + get_reg_name(arg_second)
            
        if instruction == 16:  # PUTC
            string_result = 'PUTC ' + str(arg_second)
            
        return string_result    
        
            
    def convert_program_code_to_text(self):
        self.program_text = []
        for i in np.arange(0, len(self.program_code) - (INSTRUCTION_SIZE - 1), INSTRUCTION_SIZE):
            self.program_text.append(self.convert_code_to_string(self.program_code[i],
                                              self.program_code[i + 1],
                                              self.program_code[i + 2]))
        
    def insert_labels_in_program_text(self):
        """
        В текст программы вставляет лейблы в начале необходимых строк
        Информацию берёт из self.address_to_label
        """
        for i in range(len(self.program_text)):
            if i in self.address_to_label:
                self.program_text[i] = self.address_to_label[i] + ': ' + self.program_text[i]
                
                
    def convert_putcs_to_putstr(self):
        """
        Последовательности инструкций PUTC, заканчивающиеся переводом строки заменяет одной инструкцией PUTSTR
        Всё во имя человекочитабельности!
        """
        new_program_text = []
        
        begin_putc_index = -1
        end_putc_index = -1
        string_sum = ''
        has_label = False
        
        for i in range(len(self.program_text)):
            
            line = self.program_text[i]
            
            if line.find('PUTC') == -1:  # в строке нет команды PUTC
                # предыдущая команда тоже не PUTC
                if begin_putc_index == end_putc_index:
                    new_program_text.append(line)
                else:
                    for j in range(begin_putc_index, i + 1):
                        new_program_text.append(self.program_text[j])
                begin_putc_index = i
                end_putc_index = i
                has_label = False
                continue
                
            # если в строке есть команда PUTC, а ещё встретился label
            if i in self.address_to_label:
                # оставляем все предыдущие команды в первоначальном виде
                for j in range(begin_putc_index, i):
                    new_program_text.append(self.program_text[j])
                    begin_putc_index = i
                    has_label = True
            
            # первая встреча инструкции PUTC
            if begin_putc_index == end_putc_index:  
                begin_putc_index = i
            
            current_chr = chr(int(line[line.find('PUTC') + 5:]))
            
            if current_chr == '\n':
                end_putc_index = i
                if has_label:
                    new_program_text.append(self.address_to_label[begin_putc_index]+ ': PUTSTR ' + string_sum)
                else:
                    new_program_text.append('PUTSTR ' + string_sum)
                begin_putc_index = i
                string_sum = ''
                continue
            
            string_sum = string_sum + current_chr
        
        self.program_text = new_program_text
            
    
    def write_program_text_into_file(self, disassembly_file):
        with open(disassembly_file, 'w') as outfile:
            for line in self.program_text:
                outfile.write("%s\n" % line)
                
    def disassembly(self, bytecode_file, disassembled_file):
        self.address_to_label = {}
        self.labels_counter = 0
        
        self.read_program_from_file(bytecode_file)
        self.convert_program_code_to_text()
        self.insert_labels_in_program_text()
        self.convert_putcs_to_putstr()
        self.write_program_text_into_file(disassembled_file)
        

In [11]:
dis_asm = Disassembler()
dis_asm.disassembly("fib.bin", "disasm_fib.asm")